In [1]:
from openai import OpenAI
import pandas as pd
import sqlite3
from dotenv import load_dotenv

load_dotenv()

/home/ivan/miniconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ivan/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


True

In [ ]:
filepath = "../data/Amazon-Sales-Case-Study-And-Dashboard/Amazon.csv"


## Text-to-SQL via LLM

## Data Reading and Database Creation

In [2]:
# executor
import pandas as pd

def execute_query(query, db_name='../data/Amazon-Sales-Case-Study-And-Dashboard/amazon.db', fetch=True):
    """
    Executes a given SQL query on the specified SQLite database.

    Parameters:
    query (str): The SQL query to execute.
    db_name (str): The name of the database file. Defaults to 'amazon.db'.
    fetch (bool): Whether to fetch and return query results. Defaults to True.

    Returns:
    pd.DataFrame or None: A DataFrame with query results if fetch=True, otherwise None.
    """
    try:
        # Establish connection to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(query)
        
        # Commit if the query is not a SELECT
        if not query.strip().lower().startswith("select"):
            conn.commit()
            print("Query executed successfully.")
            return None
        
        # Fetch and return results as a DataFrame
        if fetch:
            df = pd.read_sql_query(query, conn)
            return df
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return None
    
    finally:
        # Close the connection
        conn.close()

### OpenAI model(s)

True

In [3]:
system_prompt = """
You are an AI assistant specialized in generating SQL queries. Your task is to take 
natural language requests and convert them into accurate and efficient SQL statements. 

Follow these instructions carefully:
1. Understand the user's intent and the structure of the data they describe.
2. Generate SQL queries tailored to the request.
3. Use standard SQL syntax unless specified otherwise by the user.
4. If the user specifies a database system (e.g., MySQL, PostgreSQL, SQLite), adapt the query accordingly.
5. Assume the user provides table and column names as they are, unless otherwise noted.

Provide the SQL query as your response, without extra explanations unless requested.
Example:
User Query: "List all customers from the customers table who live in California."

SQL Code:
SELECT * FROM customers WHERE state = 'California'

INPUT:
1. user_query: natural language query from the user
2. database_description: optional: create table statements or text description

OUTPUT:
sql-query
Return it as a string with no leading apostrophes or smth like that
"""


def text2sql(user_query, database_description=None):
    client = OpenAI()
    
    messages =[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f'user_query: {user_query}, database_description: {database_description}'}
    ]
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7)
    
    # TODO: parse the response
    response = completion.choices[0].message.content
    
    return response

In [5]:
database_description = f"""
create table amazon_sales
(invoice_id varchar(30) primary key not null,
branch varchar(5) not null,
city varchar(30) not null,
customer_type varchar(30) not null,
gender varchar(10) not null,
product_line varchar(100) not null,
unit_price decimal(10,2) not null,
quantity int not null,
vat float not null,
total decimal(10,2) not null,
date date not null,
time time not null,
payment_method varchar(20) not null,
cogs decimal(10,2) not null,
gross_margin_percentage float not null,
gross_income decimal(10,2) not null,
rating decimal(3,1) not null,
time_of_day varchar(15) not null,
day_name varchar(10) not null,
month_name varchar(10) not null);
"""

user_query = "Which product line has the lowest sales?"
resulting_query = text2sql(user_query, database_description)
print(resulting_query)

SELECT product_line, SUM(total) AS total_sales 
FROM amazon_sales 
GROUP BY product_line 
ORDER BY total_sales ASC 
LIMIT 1;


In [18]:
dataset = pd.read_csv("../data/Amazon-Sales-Case-Study-And-Dashboard/groundtruth.csv")
dataset.head()
for _, pair in dataset.iterrows():
    nl = pair['NL']
    print(f"User query:\n{nl}")
    sql_query = pair['SQL query']
    print(f"Expected SQL:\n{sql_query}")
    result = text2sql(nl, database_description)
    print(f"Result query:\n{result}\n")
    
    expected_ex = execute_query(sql_query)
    print(f'Expected execution:\n{expected_ex}\n')
    result_ex = execute_query(result)
    print(f'Resulting execution:\n{result_ex}\n')

User query:
What is the count of distinct cities in the dataset?
Expected SQL:
select count(distinct(city)) from amazon_sales;
Result query:
SELECT COUNT(DISTINCT city) FROM amazon_sales;

Expected execution:
   count(distinct(city))
0                      3

Resulting execution:
   COUNT(DISTINCT city)
0                     3

User query:
For each branch, what is the corresponding city?
Expected SQL:
select distinct city, branch from amazon_sales;
Result query:
SELECT branch, city FROM amazon_sales;

Expected execution:
        city branch
0     Yangon      A
1  Naypyitaw      C
2   Mandalay      B

Resulting execution:
    branch       city
0        A     Yangon
1        C  Naypyitaw
2        A     Yangon
3        A     Yangon
4        A     Yangon
..     ...        ...
995      C  Naypyitaw
996      B   Mandalay
997      A     Yangon
998      A     Yangon
999      A     Yangon

[1000 rows x 2 columns]

User query:
What is the count of distinct product lines in the dataset?
Expected 

Result query:
SELECT product_line, AVG(rating) AS average_rating 
FROM amazon_sales 
GROUP BY product_line;

Expected execution:
             product_line  avg_rating
0  Electronic accessories    6.924706
1     Fashion accessories    7.029213
2      Food and beverages    7.113218
3       Health and beauty    7.003289
4      Home and lifestyle    6.837500
5       Sports and travel    6.916265

Resulting execution:
             product_line  average_rating
0  Electronic accessories        6.924706
1     Fashion accessories        7.029213
2      Food and beverages        7.113218
3       Health and beauty        7.003289
4      Home and lifestyle        6.837500
5       Sports and travel        6.916265

User query:
Count the sales occurrences for each time of day on every weekday.
Expected SQL:
select day_name, time_of_day, count(*) sales from amazon_sales group by day_name, time_of_day order by field(day_name, 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'

Result query:
SELECT day_name, AVG(rating) AS average_rating 
FROM amazon_sales 
GROUP BY day_name 
ORDER BY average_rating DESC 
LIMIT 1;

Expected execution:
  day_name  avg_rating
0        1    7.153600
1        5    7.076259
2        0    7.011278
3        2    7.003165
4        6    6.901829
5        4    6.889855
6        3    6.805594

Resulting execution:
  day_name  average_rating
0        1          7.1536

User query:
Determine the day of the week with the highest average ratings for each branch.
Expected SQL:
with avg_rating as (select branch, day_name, avg(rating) avg_rat from amazon_sales group by branch, day_name), max_rating as (select max(avg_rat) from avg_rating group by branch) select branch, day_name, avg_rat as highest_avg_rat from avg_rating where avg_rat in (select * from max_rating);
Result query:
SELECT day_name, branch, AVG(rating) AS average_rating
FROM amazon_sales
GROUP BY branch, day_name
ORDER BY branch, average_rating DESC;

Query executed successfully.
